In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv")

In [ ]:
#lib
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, metrics
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import copy
import math

In [ ]:
train.head(5)

In [ ]:
test.head()

In [ ]:
sub.head()

In [ ]:
#Checking Null values in training data
train.isnull().sum()

In [ ]:
# Checking Null values in test data
test.isnull().sum()

> Our data is already clean no null value present.... Wow!!

# Explore the Data

In [ ]:
print("There are {} products in {} stores in {} countries".format(len(train['product'].unique()),len(train['store'].unique()), len(train['country'].unique())))
print("Product Names: {}".format(train['product'].unique()))
print("Store Names: {}".format(train['store'].unique()))
print("Country Names: {}".format(train['country'].unique()))

### Which Product is sell more

In [ ]:
more_selling_product = train.groupby("product")["num_sold"].agg("max")
more_selling_product

### Sale & Percentange of Sale

In [ ]:
how_much_sold = pd.DataFrame(train.groupby("product")["num_sold"].sum()).reset_index()
# print(how_much_sold)
sale_percentage = round(how_much_sold['num_sold']/how_much_sold['num_sold'].sum()*100,2)
# print(sale_percentage)
how_much_sold['sale_per'] = sale_percentage
how_much_sold

# Feature Engineering

We have time series data that's why we insight on each country, I mean Holiday!! of each coutnry 

In [ ]:
all_data = pd.concat([train, test], axis=0, ignore_index=True)
all_data

In [ ]:
all_data['date'] = pd.to_datetime(all_data['date'])
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['day'] = all_data['date'].dt.day
all_data['dayofweek'] = all_data['date'].dt.dayofweek
all_data['dayofmonth'] = all_data['date'].dt.days_in_month
all_data['dayofyear'] = all_data['date'].dt.dayofyear
all_data['weekday'] = all_data['date'].dt.weekday
all_data['weekofyear'] = all_data['date'].dt.weekofyear

In [ ]:
all_data.head(3)

In [ ]:
all_data.drop(columns=["row_id", "date", "num_sold"], inplace=True)
all_data.head(3)

In [ ]:
le = LabelEncoder()
for col in ['country', 'store', 'product']:
    all_data[col] = le.fit_transform(all_data[col])
    
all_data.head(3)

In [ ]:
train2 = all_data[:len(train)]
test2 = all_data[len(train):]
y = train['num_sold']

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
folds = TimeSeriesSplit(n_splits=4)

In [ ]:
def smape(actual, predicted):
    numerator = np.abs(predicted - actual)
    denominator = (np.abs(actual) + np.abs(predicted)) / 2
    
    return np.mean(numerator / denominator)*100

In [ ]:
from catboost import CatBoostRegressor

y_pred = np.zeros(len(test))
scores = []

for fold, (train_id, test_id) in enumerate(folds.split(train)):
    print("Fold: ", fold)
    
    # Splitting
    X_train, y_train = train2.iloc[train_id], y.iloc[train_id]
    X_valid, y_valid = train2.iloc[test_id], y.iloc[test_id]
    

    cat = CatBoostRegressor(
        iterations=10000,
        bootstrap_type='Bayesian',
        boosting_type='Plain',
        loss_function='MAE',
        eval_metric='SMAPE'
    )

    cat.fit(
        X_train, y_train, 
        eval_set=(X_valid, y_valid),
        early_stopping_rounds=1000,
        verbose=1000
    )
    
    print('\n')
    
    # Evaluation
    valid_pred = cat.predict(X_valid)
    valid_score = smape(y_valid, np.ceil(valid_pred))
    scores.append(valid_score)
    
    # Prediction for submission
    y_pred += cat.predict(test2) / folds.n_splits

In [ ]:
score = np.array(scores).mean()
print('Mean SMAPE score: ', score)

In [ ]:
sub.num_sold = np.ceil(y_pred)
sub.head(10)

In [ ]:
sub.to_csv('submission.csv', index=False)